In [1]:
import os
import json
import csv

import re

import  ftfy

from findJsonKeys import FindJsonKey 
from populateFlatByJson import cleanText 


# transform post file
Extract the data from the facebook posts files, flat the information to the dict , join all data in a list

In [2]:
facebook_posts_dir=os.getenv("FACE_DATA")+"your_activity_across_facebook/posts/"
findKeys=FindJsonKey()
def transform(msgPost:dict)-> dict:
    result={"timestamp":msgPost["timestamp"]}
    if("title" in msgPost):
        result["title"]=cleanText(msgPost["title"])
    if("attachments" in msgPost):
        for atc in msgPost["attachments"]:
            for data in atc["data"]:
                # findKeys.makeDecision(data)
                if "event" in data:
                    pass
                if "external_context" in data:
                    pass
                if "for_sale_item" in data:
                    pass
                if "life_event" in data:
                    pass
                if "media" in data:
                    result["creation_timestamp"]=data["media"]["creation_timestamp"]
                    result["mediauri"]=data["media"]["uri"]
                    if("description" in data["media"] ):
                        result["description"]=cleanText(data["media"]["description"])
                    if("title" in data["media"] ):
                        result["mediatitle"]=cleanText(data["media"]["title"])
                if "name" in data:
                    result["name"]=cleanText(data["name"])
                if "place" in data:
                    result["address"]=cleanText(data["place"]["address"])
                    result["name"]=cleanText(data["place"]["name"])
                    result["latitude"]=data["place"]["coordinate"]["latitude"]
                    result["longitude"]=data["place"]["coordinate"]["longitude"]
                if "text" in data:
                    result["text"]=cleanText(data["text"])
    if("data" in msgPost):
        for data in msgPost["data"]:
                if "post" in data:
                    result["text"]=cleanText(data["post"])
    if("tags" in msgPost):
        for tag in msgPost["tags"]:
                    result["text"]=cleanText(tag["name"])
 
                
    return result
all_data=[]
for root,folders,files in os.walk(facebook_posts_dir):
    folder=root.replace(facebook_posts_dir,"")
    for file in [file for file in files if "your_posts" in file]:
        with open(os.path.join(root,file), 'r') as user_file:
            parsed_json = json.load(user_file)
            # findKeys.makeDecision(parsed_json)
            parsed_json = [transform(msg) for msg in parsed_json ]
            all_data += parsed_json

print(findKeys.keys())               

## After split the data to important files

with all data now need slipt the important data. I choice media , text and the post with cordinates. The same information can be into the files but I garate the file has no empty about your main data. 

In [3]:
with open(os.path.join(os.path.dirname(facebook_posts_dir),"posts.json"), 'w', newline='', encoding='utf-8') as jsonFile:                
    
    parsed_json = [item for item in all_data]
    json.dump(parsed_json,jsonFile,indent=2)         
    

with open(os.path.join(os.path.dirname(facebook_posts_dir),"postText.json"), 'w', newline='', encoding='utf-8') as jsonFile:                
    
    parsed_json = [item for item in all_data if "text" in item]
    json.dump(parsed_json,jsonFile,indent=2)      
    
with open(os.path.join(os.path.dirname(facebook_posts_dir),"postCoordinates.json"), 'w', newline='', encoding='utf-8') as jsonFile:                
    
    parsed_json = [item for item in all_data if "latitude" in item]
    json.dump(parsed_json,jsonFile,indent=2) 
    
    
with open(os.path.join(os.path.dirname(facebook_posts_dir),"postMedia.json"), 'w', newline='', encoding='utf-8') as jsonFile:                
    
    parsed_json = [item for item in all_data if "mediauri" in item]
    json.dump(parsed_json,jsonFile,indent=2) 